<a href="https://colab.research.google.com/github/JeehwanLim/202103_information-retrieval/blob/main/%EC%A4%91%EA%B0%84_%EB%A0%88%ED%8F%AC%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **환경 설정**

### **구글 드라이브 연계**

In [1]:
### Google Drive 패키지와 os 모듈 불러오기
from google.colab import drive
import os

In [2]:
### 본인 구글 드라이브의 최초 경로를 설정하기

# 대부분의 구글 드라이브 최초 경로는 아래와 같습니다. 
# 예외 발생 시 본인의 구글 드라이브에 접속하여 content 폴더나 gdrive 폴더가 어떤 위치에 있으며, 
# 자신이 연결하려는 폴더까지의 경로가 어떻게 되는지 확인해야 합니다.  
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
### 자신의 현재 경로를 파악하기

os.getcwd() ## 아마도 /content로 나올 것.

'/content'

### **엘라스틱 서버 설치 및 구동하기**

In [4]:
### 구글 클라우드 컴퓨터에 elastic server 서버 설치를 위한 폴더 생성 
!sudo mkdir /content/elasticsearch
### 접근 권한 수정
!chmod 755 -R elasticsearch
### 현재 작업 디렉토리 설정
os.chdir('/content/elasticsearch')

In [5]:
### 리눅스용 엘라스틱서치 서버 설치를 위한 패키지 다운로드
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.0.0-linux-x86_64.tar.gz -q

In [6]:
### 위에서 다운로드 받은 압축 파일을 해제
!tar -xzf elasticsearch-7.0.0-linux-x86_64.tar.gz

In [7]:
### 코랩 노트북 환경에서 서버 구동을 위해서 PPID 1의 백그라운드 데몬 프로세스가 해당 폴더에 접근이 가능하도록 소유자 변경
!chown -R daemon:daemon elasticsearch-7.0.0

In [8]:
### 한글 nori 형태소 분석기 설치 
!sudo elasticsearch-7.0.0/bin/elasticsearch-plugin install analysis-nori

-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Installed analysis-nori


In [10]:
### 파이썬 환경에서 구동을 위한 elasticsearch 패키지 설치
!pip install elasticsearch

     |████████████████████████████████| 348kB 5.6MB/s 


In [9]:
# 데몬 프로세스로 엘라스틱 서버 개시하기
import os
from subprocess import Popen, PIPE, STDOUT
es = Popen(['elasticsearch-7.0.0/bin/elasticsearch'], 
                  stdout=PIPE, stderr=STDOUT,
                  preexec_fn=lambda: os.setuid(1)  # as daemon
                )

### **엘라스틱서치 서버 구동될때까지 대기 (약 3분 소요된다고 함)**

서버가 안정적으로 열릴 때까지 시간이 조금 걸릴 수 있어요..

`es.info()`를 실행시켰을 때

아래와 같은 결과가 나오면 성공!

```
{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'EOh60gEwRri9A_UruHyQLA',
 'name': '657b2f379e3e',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2019-04-05T22:55:32.697037Z',
  'build_flavor': 'default',
  'build_hash': 'b7e28a7',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.0.0',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.7.0',
  'number': '7.0.0'}}
```



In [13]:
# 로컬 서버에 엘라스틱 서버와 python을 연결
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch("localhost:9200/")
es.info()

{'cluster_name': 'elasticsearch',
 'cluster_uuid': 'lEaeDM2vQE6Fvvw7iYLDIA',
 'name': 'd1c450b6e81d',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2019-04-05T22:55:32.697037Z',
  'build_flavor': 'default',
  'build_hash': 'b7e28a7',
  'build_snapshot': False,
  'build_type': 'tar',
  'lucene_version': '8.0.0',
  'minimum_index_compatibility_version': '6.0.0-beta1',
  'minimum_wire_compatibility_version': '6.7.0',
  'number': '7.0.0'}}

### **Elasticsearch Server 개시 및 Python과 연동**

> /content/gdrive 이후의 경로에는 구글 드라이브 보안으로 인해서
daemon process가 정상적인 방식으로는 서버 구동이 어렵기 때문에 본인 드라이브가 아닌
구글 클라우드 컴퓨터에 엘라스틱 서버를 설치하고 개시했습니다. 




In [14]:
### 현재 작업 환경 재설정하기 (자신만의 작업 디렉토리를 기본 경로로 설정하는 것)

#.ipynb 노트북 파일과 불러오려는 이미지 및 모듈에 해당하는 파일들은 같은 현재 작업 환경 
#또는 그 하위 폴더에 존재하는 것이 좋습니다. 

# 자신만의 기본 경로를 설정하는데, 대부분의 경우에는 Colab Notebooks까지 동일하게 공유하며,
# 이후 경로는 자신이 생성한 폴더에 맞추어서 경로를 설정하면 됩니다.
# 수업에서는 자신의 구글 드라이브에 information_retrieval 폴더를 생성하시면 됩니다. 
os.chdir('/content/gdrive/My Drive/Colab Notebooks/information_retrieval/')
!sudo mkdir elastic
os.chdir('/content/gdrive/My Drive/Colab Notebooks/information_retrieval/elastic')

mkdir: cannot create directory ‘elastic’: File exists


# **한국어 위키 검색**

### **문서 데이터 다운로드**

> 한국어 위키 덤프를 다운로드 받은 다음에 전처리하여 사용해보기

> abstract.xml.gz - 문서 요약 덤프 사용

https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4_%EB%8B%A4%EC%9A%B4%EB%A1%9C%EB%93%9C

In [15]:
# 샘플 영어 위키 덤프를 다운로드
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-abstract.xml.gz

--2021-05-01 06:44:14--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-abstract.xml.gz
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78294023 (75M) [application/octet-stream]
Saving to: ‘kowiki-latest-abstract.xml.gz.5’

kowiki-latest-abstr 100%[===================>]  74.67M  4.71MB/s    in 15s     

2021-05-01 06:44:30 (4.86 MB/s) - ‘kowiki-latest-abstract.xml.gz.5’ saved [78294023/78294023]



In [16]:
import gzip
import xml.etree.ElementTree as ET

input = gzip.open('kowiki-latest-abstract.xml.gz', 'r')
tree = ET.parse(input)
root = tree.getroot()

In [17]:
# root 이하 모든 자식의 태그명을 프린트한다.
for child in root.find('doc'):
  print(child.tag)

title
url
abstract
links


In [18]:
# 데이터 조회 
index = 0

for doc in root.iter("doc"):
    if index == 20:
        break
    print(doc.find('title').text)
    print(doc.find('abstract').text)

    index = index + 1

위키백과: 지미 카터
| 그림 = JimmyCarterPortrait2.jpg
위키백과: 수학
수학(數學, )은 수, 양, 구조, 공간, 변화 등의 개념을 다루는 학문이다. 널리 받아들여지는 명확한 정의는 없으나 현대 수학은 일반적으로 엄밀한 논리에 근거하여 추상적 대상을 탐구하며, 이는 규칙의 발견과 문제의 제시 및 해결의 과정으로 이루어진다.Devlin, Keith, Mathematics: The Science of Patterns: The Search for Order in Life, Mind and the Universe (Scientific American Paperback Library) 1996,  수학은 그 발전 과정에 있어서 철학, 과학과 깊은 연관을 맺고 있으며, 다만 엄밀한 논리와 특유의 추상성, 보편성에 의해 다른 학문들과 구별된다. 특히 수학은 과학의 여느 분야들과는 달리 자연계에서 관측되지 않는 개념들에 대해서까지 이론을 추상화시키는 특징을 보이는데, 수학자들은 그러한 개념들에 대한 추측을 제시하고 적절하게 선택된 정의와 공리로부터 엄밀한 연역을 거쳐 그 진위를 파�
위키백과: 수학 상수
수학에서 상수란 그 값이 변하지 않는 불변량으로, 변수의 반대말이다. 물리 상수와는 달리, 수학 상수는 물리적 측정과는 상관없이 정의된다.
위키백과: 문학
문학(文學, )은 언어를 예술적 표현의 제재로 삼아 새로운 의미를 창출하여, 인간과 사회를 진실되게 묘사하는 예술의 하위분야이다.조남현, 고등학교 문학(상), 중앙교육진흥연구소, 2003, 12~15쪽.
위키백과: 나라 목록
이 문서는 나라 목록이며, 전 세계 207개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.
위키백과: 화학
화학(化學, )은 물질의 성질, 조성, 구조, 변화 및 그에 수반하는 에너지의 변화를 연구하는 자연과학의 한 분야이다. 물리학도 역시 물질을 다루는 학문이지만, 물리학이 원소와 화합물을 모두 포함한 물체의 운동과 에너지, 열적·전기적·광학적·기계적 속성을

### **인덱스 생성**
> nori 한국어 형태소 분석기 사용 설정, 불용어 설정 등 인덱스 생성을 위한 설정

> 인덱스 생성

In [19]:
# Stop words 파일 생성 
f = open('/content/elasticsearch/elasticsearch-7.0.0/config/stopFilter.txt', 'w')

f.write("위키\n")
f.write("백과\n")
f.write("위키백과")

f.close()

In [20]:
index_settings = {
    
    "settings":{
        "number_of_shards": 5,
        "number_of_replicas": 1,
        "analysis":{
            "tokenizer":{
                "korean_nori_tokenizer":{
                    "type":"nori_tokenizer",
                    "decompound_mode":"mixed"
                    # "user_dictionary":"user_dictionary.txt"
                }
            },
            "analyzer":{
                "nori_analyzer":{
                    "type":"custom",
                    "tokenizer":"korean_nori_tokenizer",
                    "filter":[
                        "nori_posfilter",
                        "nori_readingform",
                        # "synonym_filtering",
                        "stop_filtering"
                    ]
                }
            },
            "filter":{
                "nori_posfilter":{
                    "type":"nori_part_of_speech",
                    "stoptags":[
                        "E","IC","J","MAG","MM","NA","NR","SC",
                        "SE","SF","SH","SL","SN","SP","SSC","SSO",
                        "SY","UNA","UNKNOWN","VA","VCN","VCP","VSV",
                        "VV","VX","XPN","XR","XSA","XSN","XSV"
                    ]
                },
                # "synonym_filtering":{
                #     "type":"synonym"
                #     ,"synonyms_path":"synonymsFilter.txt"
                # },
                "stop_filtering":{
                    "type":"stop",
                    "stopwords_path":"stopFilter.txt"
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "title": {
                "type": "text",
                "analyzer": "nori_analyzer"
            },
            "abstract": {
                "type": "text",
                "analyzer": "nori_analyzer"
            }
        }
    }
}

In [21]:
# 데이터를 색인화하기 위한 함수
def indexing(es, index_name):
    # 이미 존재할 경우 삭제하고 다시 만들기
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)

    # 인덱스 생성
    print(es.indices.create(index=index_name, body=index_settings))

# 인덱스명을 정하기 (자유롭게)
index_name = 'korwiki_abs_index'
indexing(es, index_name)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'korwiki_abs_index'}


### **한글 위키 압축 문서로 인덱싱**

> 인덱싱하는 시간을 단축하기 위해 helpers.bulk 사용

In [22]:
# 인덱스 생성 
docs = []

for doc in root.iter("doc"):
    dic = {}
    str = doc.find('title').text
    dic['title'] = str.replace("위키백과:","")
    dic['abstract'] = doc.find('abstract').text

    docs.append({
        '_index': index_name,
        '_source': dic })

# es.index(index=index_name, doc_type='string', body=dic)
helpers.bulk(es, docs)

# 인덱스 리프레쉬 
es.indices.refresh(index=index_name)

{'_shards': {'failed': 0, 'successful': 5, 'total': 10}}

### **색인화한 검색 엔진에서 정보 검색**

> es.search를 활용해서 검색해보기

```
es.search(index = '선언한 인덱스명', body = {'from': '몇 위부터 반환할 것인지' , 
'size': '최대 반환할 갯수', 'query' : {'match':{'기준 key' : '검색할 내용'}}})
```

### 결과로는 무엇이 반환되었을까...? 

> 딕셔너리 형태로, 필요한 정보는 딕셔너리 value에 대한 접근 방식을 통해 얻을 수 있습니다.

```
{'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 
'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'},
'max_score': 1.2037696, 'hits': [{'_index': 'sample_index', '_type': 'string',
'_id': 'zYMJ9HcB0qdz_4P6yRNV', '_score': 1.2037696, '_source': 
{'songs': '젊은그대', 'lyrics': '사랑스런 젊은 그대 태양같은 젊은 그대'}}, 
{'_index': 'sample_index', '_type': 'string', '_id': 'yYMJ9HcB0qdz_4P6yBPd',
'_score': 1.0137007, '_source': {'songs': '민족의 아리아', 'lyrics': '지축을 박차고 
포효하라 그대'}}]}}
```

> 딕셔너리의 접근은..?

```
딕셔너리 변수명['key'] # value를 반환! 
```




In [23]:
# 검색 해보기
def search(keyword=None, size=5):
    body = {
        "from":0,
        "size": size,
        "query":{
            "multi_match":{
                "query": keyword,
                "fields": ['title','abstract']
            }
        }
    }

    res = es.search(index=index_name, body=body)
    return res

results = search('고려대', 10)
# results = es.search(index=index_name, body={'from':0, 'size':10, 'query':{'match':{'abstract':'위키백과'}}})

# 딕셔너리 안에 딕셔너리: 'hits'라는 key 안에 value는 또 다른 사전형으로 되어있으며, 해당 사전의 key 이름은 'hits'   
for result in results['hits']['hits']:
    print('score:', result['_score'], 'source::', result['_source'])

score: 25.427958 source:: {'title': ' 고려대로', 'abstract': '고려대로(高麗大路)는 서울 성북구의 도로이다. 본래 도로명은 인촌로(仁村路)였으나 친일 논란으로 개칭되었다.'}
score: 21.864128 source:: {'title': ' 고려대학교안암병원', 'abstract': '|주소      = 서울특별시 성북구 고려대로 73 (안암동5가)'}
score: 21.855015 source:: {'title': ' 권문혁', 'abstract': '권문혁(1962년 7월 10일 ~ )은 대한민국의 언론인이다. 고려대'}
score: 21.855015 source:: {'title': ' 서울안암초등학교', 'abstract': '|위치 = 서울특별시 성북구 고려대로7가길 39'}
score: 21.71157 source:: {'title': ' 40080년', 'abstract': '《40080년》은 1995년 고려대 수학과 94학번인 이민식이 제작한 횡스크롤 슈팅 게임이다.'}
score: 21.051743 source:: {'title': ' 용문중학교 (서울)', 'abstract': '|위치     = 서울특별시 성북구 고려대로17가길 46 (안암동2가)'}
score: 20.942371 source:: {'title': ' 동광제약', 'abstract': '|장소 = 서울특별시 중구 퇴계로 97 고려대연각타워 12층'}
score: 19.464134 source:: {'title': ' 고려대연각타워', 'abstract': '서울특별시 중구 충무로1가 25-5 (퇴계로 97 )'}
score: 17.40315 source:: {'title': ' 대한법의학회', 'abstract': '대한법의학회(The Korean Society For Legal Medicine)는 1976년에 설립된 대한민국의 의학 단체이다. 소재지는 서울특별시 성북구 고려대로 73, 고려대학교 의과대학 법의학교실에 위치하고 있다